# many-to-one

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
words = ['bad', 'nasty', 'foul', 'awful', 'disgusting', 'offensive', 'horrible', 'good', 'fine', 'nice', 'poor', 'wrong']
y_data = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0] # 부정은 0, 긍정은 1, 전체 12개 데이터

In [3]:
char_set = ['<pad>'] + sorted(list(set(''.join(words))))
idx2char = {idx : char for idx, char in enumerate(char_set)} 
char2idx = {char : idx for idx, char in enumerate(char_set)} 
print(char_set) 
print(idx2char) 
print(char2idx)

['<pad>', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'l', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y']
{0: '<pad>', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'l', 11: 'n', 12: 'o', 13: 'p', 14: 'r', 15: 's', 16: 't', 17: 'u', 18: 'v', 19: 'w', 20: 'y'}
{'<pad>': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'l': 10, 'n': 11, 'o': 12, 'p': 13, 'r': 14, 's': 15, 't': 16, 'u': 17, 'v': 18, 'w': 19, 'y': 20}


In [4]:
X_data = list(map(lambda word : [char2idx.get(char) for char in word], words)) 
print(X_data)

[[2, 1, 4], [11, 1, 15, 16, 20], [6, 12, 17, 10], [1, 19, 6, 17, 10], [4, 9, 15, 7, 17, 15, 16, 9, 11, 7], [12, 6, 6, 5, 11, 15, 9, 18, 5], [8, 12, 14, 14, 9, 2, 10, 5], [7, 12, 12, 4], [6, 9, 11, 5], [11, 9, 3, 5], [13, 12, 12, 14], [19, 14, 12, 11, 7]]


In [5]:
max_sequence = 10

for x in X_data:
  x += [0] * (max_sequence-len(x))
print(X_data)

[[2, 1, 4, 0, 0, 0, 0, 0, 0, 0], [11, 1, 15, 16, 20, 0, 0, 0, 0, 0], [6, 12, 17, 10, 0, 0, 0, 0, 0, 0], [1, 19, 6, 17, 10, 0, 0, 0, 0, 0], [4, 9, 15, 7, 17, 15, 16, 9, 11, 7], [12, 6, 6, 5, 11, 15, 9, 18, 5, 0], [8, 12, 14, 14, 9, 2, 10, 5, 0, 0], [7, 12, 12, 4, 0, 0, 0, 0, 0, 0], [6, 9, 11, 5, 0, 0, 0, 0, 0, 0], [11, 9, 3, 5, 0, 0, 0, 0, 0, 0], [13, 12, 12, 14, 0, 0, 0, 0, 0, 0], [19, 14, 12, 11, 7, 0, 0, 0, 0, 0]]


In [6]:
X_data = torch.Tensor(X_data)
X_data = X_data.unsqueeze(2)
y_data = torch.LongTensor(y_data)

print(X_data.shape)
print(y_data.shape)

torch.Size([12, 10, 1])
torch.Size([12])


In [7]:
batch_size = X_data.shape[0]
seq_length = max_sequence
input_size = 1
hidden_size = 4 
output_size = 2

In [8]:
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.rnn = nn.GRU(input_size=input_size, hidden_size=hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        batch_size = x.shape[0]
        hidden = torch.zeros(1, batch_size, self.hidden_size) # 1 = num_layers * n_directions
        x, hidden = self.rnn(x, hidden)
        x = x[:,-1,:]
        output = self.fc(x)
        return output

In [9]:
import numpy as np

model = SimpleRNN(input_size, hidden_size, output_size)

with torch.no_grad():
    y = model(X_data)
    print(y)
    print(np.argmax(y.cpu(), axis=1), y_data)

tensor([[ 0.1417,  0.7098],
        [ 0.1360,  0.6832],
        [ 0.1426,  0.6967],
        [ 0.1371,  0.6851],
        [-0.6458,  0.0080],
        [-0.2379,  0.3085],
        [-0.0075,  0.5167],
        [ 0.1454,  0.7002],
        [ 0.1458,  0.6998],
        [ 0.1474,  0.7024],
        [ 0.1407,  0.6953],
        [ 0.1332,  0.6798]])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]) tensor([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0])


In [10]:
# RNN 학습

model = SimpleRNN(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = 0.1)

for epoch in range(500):
    optimizer.zero_grad()
    loss = 0

    model.train()
    y_pred = model(X_data)
    loss = criterion(y_pred, y_data) 

    if epoch % 100 == 0:
        model.eval()
        with torch.no_grad():
          idx_pred = np.array(np.argmax(y_pred, axis=1))
          print(idx_pred)
          correct = (idx_pred == np.array(y_data)).sum()
          accuracy = correct * 100 / batch_size
        print("epoch: %d, loss: %1.3f, accuracy: %f" % (epoch+1,loss.data, accuracy))
    loss.backward()
    optimizer.step()


[0 0 0 0 1 1 1 0 0 0 0 0]
epoch: 1, loss: 0.739, accuracy: 50.000000
[0 0 0 0 0 0 0 1 1 1 0 0]
epoch: 101, loss: 0.000, accuracy: 100.000000
[0 0 0 0 0 0 0 1 1 1 0 0]
epoch: 201, loss: 0.000, accuracy: 100.000000
[0 0 0 0 0 0 0 1 1 1 0 0]
epoch: 301, loss: 0.000, accuracy: 100.000000
[0 0 0 0 0 0 0 1 1 1 0 0]
epoch: 401, loss: 0.000, accuracy: 100.000000
